# AutoML: Train "the best" Time-Series Forecasting model for Retail Dataset.

# 1. Connect to Azure ML Workspace

In [ ]:
import warnings
import logging

# Suppress OpenTelemetry warnings
warnings.filterwarnings("ignore", message="Overriding of current")
warnings.filterwarnings("ignore", message="Attempting to instrument")

# Suppress Azure SDK telemetry logging
logging.getLogger("azure.core.pipeline.policies.http_logging_policy").setLevel(logging.WARNING)
logging.getLogger("azure.identity").setLevel(logging.WARNING)
logging.getLogger("opentelemetry").setLevel(logging.ERROR)

In [ ]:
# Import required libraries
from azure.ai.ml import MLClient

from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import automl
from azure.ai.ml import Input

In [ ]:
from azure.identity import AzureCliCredential

credential = AzureCliCredential()
ml_client = None
try:
    subscription_id = "57123c17-af1a-4ec2-9494-a214fb148bf4"
    resource_group = "admin-rg"
    workspace = "ml-demo-wksp-wus-01"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)
except Exception as ex:
    print("Ex:", ex)

In [ ]:
# Verify connection
ws = ml_client.workspaces.get(ml_client.workspace_name)
print(f"Connected to: {ws.name} ({ws.location})")

# 2. Data Preparation

Using [Retail data analytics](https://www.kaggle.com/datasets/manjeetsingh/retaildataset) - weekly sales by store and department.

## 2.1 Load Datasets


In [ ]:
import pandas as pd

# Load datasets
stores_df = pd.read_csv('../dataset/stores data-set.csv')
features_df = pd.read_csv('../dataset/Features data set.csv')
sales_df = pd.read_csv('../dataset/sales data-set.csv')

# Quick exploration
print(f"Stores: {stores_df.shape}")
print(f"Features: {features_df.shape}")
print(f"Sales: {sales_df.shape}")

print("\n--- Stores Data ---")
display(stores_df.head())

print("\n--- Features Data ---")
display(features_df.head())

print("\n--- Sales Data ---")
display(sales_df.head())


## 2.2 Merge Datasets
Merge sales with stores (on Store) and then with features (on Store and Date).



In [ ]:
# Merge sales with stores (on Store)
merged_df = sales_df.merge(stores_df, on='Store', how='left')

# Merge with features (on Store and Date)
merged_df = merged_df.merge(features_df, on=['Store', 'Date'], how='left', suffixes=('', '_feat'))

# Drop duplicate IsHoliday column from features
merged_df = merged_df.drop(columns=['IsHoliday_feat'])

print(f"Merged dataset shape: {merged_df.shape}")
print(f"\nColumns: {merged_df.columns.tolist()}")
display(merged_df.head())


## 2.3 Feature Engineering
Create new features from date, handle missing MarkDown values, and encode categorical variables.



In [ ]:
# Convert Date to datetime (format is dd/mm/yyyy)
merged_df['Date'] = pd.to_datetime(merged_df['Date'], dayfirst=True)

# Extract date features
merged_df['Year'] = merged_df['Date'].dt.year
merged_df['Month'] = merged_df['Date'].dt.month
merged_df['Week'] = merged_df['Date'].dt.isocalendar().week
merged_df['DayOfWeek'] = merged_df['Date'].dt.dayofweek

# Handle missing MarkDown values (only available after Nov 2011)
markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
merged_df[markdown_cols] = merged_df[markdown_cols].fillna(0)

# Encode categorical: Store Type (A, B, C)
if 'Type' in merged_df.columns:
    merged_df = pd.get_dummies(merged_df, columns=['Type'], prefix='StoreType')

print(f"Feature engineered dataset: {merged_df.shape}")
display(merged_df.head())


## 2.4 Time-Based Train/Validation Split

Split data chronologically: train on data before 2012, validate on 2012 data.


In [ ]:
# Check for duplicates
dupes = merged_df.groupby(['Store', 'Dept', 'Date']).size()
print(f"Duplicate combinations: {(dupes > 1).sum()}")

# Aggregate duplicates: sum Weekly_Sales, take first for other columns
agg_funcs = {col: 'first' for col in merged_df.columns if col not in ['Store', 'Dept', 'Date']}
agg_funcs['Weekly_Sales'] = 'sum'
merged_df = merged_df.groupby(['Store', 'Dept', 'Date'], as_index=False).agg(agg_funcs)
print(f"After deduplication: {merged_df.shape}")

# Sort by date
merged_df = merged_df.sort_values(['Store', 'Dept', 'Date'])

# Time-based split: train on data before 2012, validate on 2012
train_df = merged_df[merged_df['Year'] < 2012].copy()
validation_df = merged_df[merged_df['Year'] >= 2012].copy()

print(f"\nTraining set: {train_df.shape}")
print(f"Validation set: {validation_df.shape}")
print(f"Train date range: {train_df['Date'].min()} to {train_df['Date'].max()}")
print(f"Validation date range: {validation_df['Date'].min()} to {validation_df['Date'].max()}")


## 2.5 Prepare Data for Azure ML AutoML

Rename columns to match AutoML expectations and save as MLTable format.


In [ ]:
import os

# Rename columns for AutoML compatibility
train_df = train_df.rename(columns={'Weekly_Sales': 'demand', 'Date': 'timeStamp'})
validation_df = validation_df.rename(columns={'Weekly_Sales': 'demand', 'Date': 'timeStamp'})

# Create single time series ID column (before converting dates to strings)
train_df['ts_id'] = train_df['Store'].astype(str) + '_' + train_df['Dept'].astype(str)
validation_df['ts_id'] = validation_df['Store'].astype(str) + '_' + validation_df['Dept'].astype(str)

# ============================================================================
# STEP 1: Filter to common time series IDs (both train and validation must have same IDs)
# ============================================================================
train_ts_ids = set(train_df['ts_id'].unique())
val_ts_ids = set(validation_df['ts_id'].unique())

val_only_ids = val_ts_ids - train_ts_ids
train_only_ids = train_ts_ids - val_ts_ids
common_ids = train_ts_ids & val_ts_ids

print(f"=== Step 1: Filter to common time series ===")
print(f"Time series in training only: {len(train_only_ids)}")
print(f"Time series in validation only: {len(val_only_ids)}")
print(f"Common time series: {len(common_ids)}")

# Keep only common IDs
train_df = train_df[train_df['ts_id'].isin(common_ids)]
validation_df = validation_df[validation_df['ts_id'].isin(common_ids)]

# ============================================================================
# STEP 2: Check contiguity - validation must start right after training ends
# For weekly data, the gap should be exactly 7 days
# ============================================================================
print(f"\n=== Step 2: Check contiguity (no gaps between train and validation) ===")

# Get max date per ts_id in training
train_max_dates = train_df.groupby('ts_id')['timeStamp'].max().reset_index()
train_max_dates.columns = ['ts_id', 'train_max_date']

# Get min date per ts_id in validation
val_min_dates = validation_df.groupby('ts_id')['timeStamp'].min().reset_index()
val_min_dates.columns = ['ts_id', 'val_min_date']

# Merge to compare
contiguity_check = train_max_dates.merge(val_min_dates, on='ts_id')
contiguity_check['train_max_date'] = pd.to_datetime(contiguity_check['train_max_date'])
contiguity_check['val_min_date'] = pd.to_datetime(contiguity_check['val_min_date'])
contiguity_check['gap_days'] = (contiguity_check['val_min_date'] - contiguity_check['train_max_date']).dt.days

# For weekly data, gap should be 7 days (next week)
# Allow some flexibility: 6-8 days is acceptable
contiguity_check['is_contiguous'] = contiguity_check['gap_days'].between(6, 8)

non_contiguous = contiguity_check[~contiguity_check['is_contiguous']]
contiguous_ids = set(contiguity_check[contiguity_check['is_contiguous']]['ts_id'])

print(f"Contiguous time series: {len(contiguous_ids)}")
print(f"Non-contiguous time series (will be removed): {len(non_contiguous)}")

if len(non_contiguous) > 0:
    print(f"\nSample non-contiguous series:")
    sample = non_contiguous.head(10)
    for _, row in sample.iterrows():
        print(f"  {row['ts_id']}: train ends {row['train_max_date'].date()}, val starts {row['val_min_date'].date()} (gap: {row['gap_days']} days)")

# Filter to only contiguous time series
train_df = train_df[train_df['ts_id'].isin(contiguous_ids)]
validation_df = validation_df[validation_df['ts_id'].isin(contiguous_ids)]

print(f"\n=== Final Dataset ===")
print(f"Training time series: {train_df['ts_id'].nunique()}")
print(f"Validation time series: {validation_df['ts_id'].nunique()}")
print(f"Training rows: {len(train_df)}")
print(f"Validation rows: {len(validation_df)}")

# Convert timestamp to consistent date string format (no time component)
train_df['timeStamp'] = pd.to_datetime(train_df['timeStamp']).dt.strftime('%Y-%m-%d')
validation_df['timeStamp'] = pd.to_datetime(validation_df['timeStamp']).dt.strftime('%Y-%m-%d')

# Verify no duplicates
train_dupes = train_df.duplicated(subset=['ts_id', 'timeStamp']).sum()
val_dupes = validation_df.duplicated(subset=['ts_id', 'timeStamp']).sum()
print(f"Train duplicates: {train_dupes}, Validation duplicates: {val_dupes}")

# Create output directories
os.makedirs('./data/training-mltable-folder', exist_ok=True)
os.makedirs('./data/validation-mltable-folder', exist_ok=True)

# Save as CSV (MLTable will reference these)
train_df.to_csv('./data/training-mltable-folder/train.csv', index=False)
validation_df.to_csv('./data/validation-mltable-folder/validation.csv', index=False)

print(f"\nTraining data saved: {len(train_df)} rows")
print(f"Validation data saved: {len(validation_df)} rows")


In [ ]:
train_df.head()

In [ ]:
validation_df.head()

In [ ]:
mltable_train = """paths:
  - file: ./train.csv
transformations:
  - read_delimited:
      delimiter: ','
      header: all_files_same_headers
"""

mltable_val = """paths:
  - file: ./validation.csv
transformations:
  - read_delimited:
      delimiter: ','
      header: all_files_same_headers
"""

with open('./data/training-mltable-folder/MLTable', 'w') as f:
    f.write(mltable_train)
    
with open('./data/validation-mltable-folder/MLTable', 'w') as f:
    f.write(mltable_val)

print("MLTable files created:")
print("  - ./data/training-mltable-folder/MLTable")
print("  - ./data/validation-mltable-folder/MLTable")


## 2.6 Upload Data to Azure Blob Storage

Due to Azure Policy restrictions (SAS tokens disabled), data must be uploaded using Azure CLI with OAuth authentication.


In [ ]:
import subprocess

# Azure Storage configuration
STORAGE_ACCOUNT = "mldemowkspwus02609576373"
CONTAINER = "azureml-blobstore-cff56e3a-d016-4526-aa58-71c460675066"

def upload_to_blob(source_folder, destination_path):
    """Upload local folder to Azure Blob Storage using OAuth authentication."""
    # First, delete existing data to ensure fresh upload
    delete_cmd = [
        "az", "storage", "blob", "delete-batch",
        "--account-name", STORAGE_ACCOUNT,
        "--source", CONTAINER,
        "--pattern", f"{destination_path}/*",
        "--auth-mode", "login"
    ]
    print(f"Cleaning {destination_path}...")
    subprocess.run(delete_cmd, capture_output=True, text=True)
    
    # Upload new data
    upload_cmd = [
        "az", "storage", "blob", "upload-batch",
        "--account-name", STORAGE_ACCOUNT,
        "--destination", CONTAINER,
        "--destination-path", destination_path,
        "--source", source_folder,
        "--auth-mode", "login",
        "--overwrite"
    ]
    print(f"Uploading {source_folder} to {destination_path}...")
    result = subprocess.run(upload_cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✓ Successfully uploaded to {destination_path}")
        print(f"  Output: {result.stdout[:200] if result.stdout else 'OK'}")
    else:
        print(f"✗ Upload failed: {result.stderr}")
    return result.returncode == 0

# Upload training data to NEW path
upload_to_blob("./data/training-mltable-folder", "retail-train-v2")

# Upload validation data to NEW path
upload_to_blob("./data/validation-mltable-folder", "retail-val-v2")

print("\nData upload complete!")


In [ ]:
my_training_data_input = Input(
    type=AssetTypes.MLTABLE, 
    path="azureml://datastores/workspaceblobstore_identity/paths/retail-train-v2"
)

my_validation_data_input = Input(
    type=AssetTypes.MLTABLE, 
    path="azureml://datastores/workspaceblobstore_identity/paths/retail-val-v2"
)

# 3. Configure and Run AutoML Forecasting Job

## 3.1 Job Configuration

In [ ]:
# Create the AutoML forecasting job with the related factory-function.
forecasting_job = automl.forecasting(
    experiment_name="sales-forecasting-v2",
    compute="teslat4-gpu-wus",  
    training_data=my_training_data_input,
    validation_data=my_validation_data_input, 
    target_column_name="demand",
    primary_metric="NormalizedRootMeanSquaredError",
    enable_model_explainability=True,
    tags={"retail": "forecasting"},
)

# Limits are all optional
forecasting_job.set_limits(
    timeout_minutes=600,
    trial_timeout_minutes=20,
    max_trials=5,
    enable_early_termination=True,
)

# Specialized properties for Time Series Forecasting training
forecasting_job.set_forecast_settings(
    time_column_name="timeStamp",
    forecast_horizon=12,  # 12 weeks forecast
    frequency="W-FRI",  # pandas offset: W-FRI=weekly anchored on Friday (matches our data)
    time_series_id_column_names=["ts_id"],
    short_series_handling_config="auto",  # Auto-handle short/irregular series
    target_lags="auto",
)

# forecasting_job.set_training(blocked_training_algorithms=["ExtremeRandomTrees"])

## 3.2 Submit Job

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(forecasting_job)
print(f"Created job: {returned_job}")

In [ ]:
ml_client.jobs.stream(returned_job.name)

# 4. Register the model


In [ ]:
# ============================================================================
# NOTE: The model requires Azure ML runtime which doesn't work on Apple Silicon.
# Instead, we'll register the model and run batch inference in Azure ML.
# ============================================================================

from azure.ai.ml.entities import Model

# Register the best model in Azure ML Model Registry
print("Registering model in Azure ML...")

model = Model(
    path=f"azureml://jobs/{returned_job.name}/outputs/best_model",
    name="retail-sales-forecasting-model",
    description="AutoML time-series forecasting model for retail weekly sales",
    type="mlflow_model"
)

try:
    registered_model = ml_client.models.create_or_update(model)
    print(f"✓ Registered model: {registered_model.name}, version: {registered_model.version}")
except Exception as e:
    print(f"Model may already be registered: {e}")
    # Get existing model
    registered_model = ml_client.models.get(name="retail-sales-forecasting-model", version="latest")


# 5. Deploy and Test the Model

Deploy the model as a Managed Online Endpoint to test predictions.


In [61]:
# ============================================================================
# Step 1: Create a Managed Online Endpoint
# ============================================================================
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment

import datetime

# Create a unique endpoint name
endpoint_name = f"retail-forecast-{datetime.datetime.now().strftime('%m%d%H%M')}"

# Define the endpoint
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Retail sales forecasting endpoint",
    auth_mode="key"  # or "aml_token" for Azure AD auth
)

# Create the endpoint (this takes a few minutes)
print(f"Creating endpoint: {endpoint_name}")
print("This may take 5-10 minutes...")

ml_client.online_endpoints.begin_create_or_update(endpoint).result()
print(f"✓ Endpoint created: {endpoint_name}")


Creating endpoint: retail-forecast-12022118
This may take 5-10 minutes...
✓ Endpoint created: retail-forecast-12022118


In [ ]:
# ============================================================================
# Step 2: Deploy the Model to the Endpoint
# ============================================================================

# Get the registered model
model = ml_client.models.get(name="retail-sales-forecasting-model", version="1")
print(f"Using model: {model.name}, version: {model.version}")

# Create the deployment with extended timeout (model takes ~90s for first inference)
from azure.ai.ml.entities import OnlineRequestSettings

deployment = ManagedOnlineDeployment(
    name="default",
    endpoint_name=endpoint_name,
    model=model,
    instance_type="Standard_DS3_v2",  # 4 cores, 14 GB RAM
    instance_count=1,
    request_settings=OnlineRequestSettings(
        request_timeout_ms=180000,  # 3 minutes timeout
        max_concurrent_requests_per_instance=1
    )
)

# Deploy (this takes 5-15 minutes)
print(f"Deploying model to endpoint...")
print("This may take 10-15 minutes...")

ml_client.online_deployments.begin_create_or_update(deployment).result()

# Set the deployment to receive 100% of traffic
endpoint.traffic = {"default": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

print(f"✓ Model deployed to: {endpoint_name}")


Check: endpoint retail-forecast-12022118 exists


Using model: retail-sales-forecasting-model, version: 1
Deploying model to endpoint...
This may take 10-15 minutes...
...............................................................................................✓ Model deployed to: retail-forecast-12022118


In [77]:
# ============================================================================
# Step 3: Test the Endpoint with Sample Data
# ============================================================================
import json
import numpy as np

# Load validation data
test_df = pd.read_csv('./data/validation-mltable-folder/validation.csv')

# Get data for one time series
ts_data = test_df[test_df['ts_id'] == '1_1'].copy().reset_index(drop=True)

# ============================================================================
# KEY INSIGHT: AutoML forecasting needs:
#   - Historical rows WITH actual 'demand' values (for context/lags)
#   - Future rows WITH demand = NaN (rows we want to predict)
# ============================================================================

# Use first 5 rows as HISTORY (with actual values)
# Use next 2 rows as FUTURE (set demand to NaN - these will be predicted)
history = ts_data.head(5).copy()
future = ts_data.iloc[5:7].copy()

# Store actual values for comparison
actual_demand = future['demand'].tolist()

# Set future demand to NaN - this tells the model to forecast these
future['demand'] = np.nan

# Combine history + future
forecast_input = pd.concat([history, future], ignore_index=True)

print("=== INPUT DATA FOR FORECASTING ===")
print(f"\nHistory (5 rows with actual demand):")
display(history[['timeStamp', 'demand', 'ts_id']].head())

print(f"\nFuture (2 rows with demand=NaN - to be predicted):")
display(future[['timeStamp', 'demand', 'ts_id']])

print(f"\nActual values we want to predict: {actual_demand}")

# AutoML expects pandas-split format
# Convert NaN to None for JSON serialization (NaN is not valid JSON)
def nan_to_none(val):
    """Convert numpy NaN to Python None for JSON compatibility."""
    if isinstance(val, float) and np.isnan(val):
        return None
    return val

# Convert data, replacing NaN with None
data_cleaned = [[nan_to_none(v) for v in row] for row in forecast_input.values.tolist()]

input_data = {
    "input_data": {
        "columns": forecast_input.columns.tolist(),
        "index": list(range(len(forecast_input))),
        "data": data_cleaned
    }
}

# Save to file (None becomes null in JSON, which AutoML understands as missing)
with open("./sample_request.json", "w") as f:
    json.dump(input_data, f, indent=2)

print(f"\n✓ Request saved with {len(history)} history rows + {len(future)} forecast rows")


=== INPUT DATA FOR FORECASTING ===

History (5 rows with actual demand):


,timeStamp,demand,ts_id
0,2012-01-06,16567.69,1_1
1,2012-01-13,16894.40,1_1
2,2012-01-20,18365.10,1_1
3,2012-01-27,18378.16,1_1
4,2012-02-03,23510.49,1_1



Future (2 rows with demand=NaN - to be predicted):


,timeStamp,demand,ts_id
5,2012-02-10,NaN,1_1
6,2012-02-17,NaN,1_1



Actual values we want to predict: [36988.49, 54060.1]

✓ Request saved with 5 history rows + 2 forecast rows


In [80]:
# ============================================================================
# Step 4: Invoke the Endpoint to Get Predictions
# ============================================================================
import requests

print(f"Calling endpoint: {endpoint_name}")
print("=" * 50)

# The model takes ~22 seconds. Use requests with longer timeout.
# Get endpoint details
endpoint_info = ml_client.online_endpoints.get(endpoint_name)
scoring_uri = endpoint_info.scoring_uri
api_key = ml_client.online_endpoints.get_keys(endpoint_name).primary_key

print(f"Scoring URI: {scoring_uri}")
print("Using 60-second timeout...")

try:
    # Read the request file
    with open("./sample_request.json", "r") as f:
        request_data = json.load(f)
    
    # Call with extended timeout
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    
    resp = requests.post(scoring_uri, json=request_data, headers=headers, timeout=60)
    resp.raise_for_status()
    response = resp.text

    print(f"Raw response: {response[:500]}")
    
    # Parse the response
    predictions = json.loads(response)
    print("\n📊 PREDICTIONS:")
    print("=" * 50)

    # Handle different response formats
    if isinstance(predictions, list):
        pred_values = predictions
    elif isinstance(predictions, dict):
        pred_values = predictions.get('predictions', predictions.get('forecast', list(predictions.values())[0]))
    else:
        pred_values = [predictions]
    
    print(f"Predicted values: {pred_values}")
    print(f"Actual values:    {actual_demand}")
    
    # Calculate error
    if len(pred_values) == len(actual_demand):
        for i, (pred, actual) in enumerate(zip(pred_values, actual_demand)):
            error = actual - pred
            pct_error = (error / actual) * 100 if actual != 0 else 0
            print(f"  Row {i+1}: Predicted={pred:.2f}, Actual={actual:.2f}, Error={error:.2f} ({pct_error:.1f}%)")

except Exception as e:
    print(f"\n❌ Error: {e}")
    print("\n📋 Getting logs...")
    
    logs = ml_client.online_deployments.get_logs(
        name="default",
        endpoint_name=endpoint_name,
        lines=100
    )
    print(logs[-2000:])


Calling endpoint: retail-forecast-12022118
Scoring URI: https://retail-forecast-12022118.westus.inference.ml.azure.com/score
Using 60-second timeout...

❌ Error: 408 Client Error: Request Timeout for url: https://retail-forecast-12022118.westus.inference.ml.azure.com/score

📋 Getting logs...
 [03/Dec/2025:02:40:03 +0000] "GET / HTTP/1.0" 200 7 "-" "kube-probe/1.29"
2025-12-03 02:40:13,664 I [71] gunicorn.access - 127.0.0.1 - - [03/Dec/2025:02:40:13 +0000] "GET / HTTP/1.0" 200 7 "-" "kube-probe/1.29"
2025-12-03 02:40:13,665 I [71] gunicorn.access - 127.0.0.1 - - [03/Dec/2025:02:40:13 +0000] "GET / HTTP/1.0" 200 7 "-" "kube-probe/1.29"
/azureml-envs/azureml-automl/lib/python3.9/site-packages/azureml/training/tabular/models/forecasting_pipeline_wrapper_base.py:1596: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for g